<a href="https://colab.research.google.com/github/1184022880com/TEST/blob/master/mod_clf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
drive.mount('/gdrive')
os.symlink('/gdrive/My Drive', '/content/gdrive')
!ls -l /content/gdrive/

In [ ]:
%matplotlib inline
import os,random
os.environ["KERAS_BACKEND"] = "tensorflow"
import tensorflow as tf
import numpy as np
from keras.utils import np_utils
import keras.models as models
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten
from keras.layers.noise import GaussianNoise
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.regularizers import *
from keras.optimizers import *
import matplotlib.pyplot as plt
import seaborn as sns
import  random, sys, keras

In [ ]:
import pickle as pk
f=open('/content/gdrive/RML2016.10a_dict.pkl','rb')
Xd=pk.load(f,encoding='latin1')
#snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])
mods,snrs = [sorted(list(set([k[j] for k in Xd.keys()]))) for j in [0,1] ]
X = []  
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]):  lbl.append((mod,snr))
X = np.vstack(X)

In [ ]:
np.random.seed(2016)
n_examples = X.shape[0]
n_train = n_examples * 0.5
train_idx = np.random.choice(range(0,n_examples), size=int(n_train), replace=False)
test_idx = list(set(range(0,n_examples))-set(train_idx))
X_train = X[train_idx]
X_test =  X[test_idx]

def to_onehot(yy):
    yy1 = np.zeros([len(yy), len(mods)])#map可以强制转换成list，这样可以求len
    yy1[np.arange(len(yy)), yy] = 1#arange创建长度指定的等差数列
    return yy1
Y_train = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), train_idx)))
Y_test = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), test_idx)))

In [ ]:
classes = mods
in_shp = list(X_train.shape[1:])
dr = 0.5 # dropout rate取0.5~0.3可以较好顾及鲁棒性，同时不丢掉重要特征。也不会出现过拟合
model = models.Sequential()
model.add(Reshape([1]+in_shp, input_shape=in_shp))
model.add(ZeroPadding2D((0, 2)))
# stride ，在 2.0 中要用 （） 包起来,同时padding选择same，保证输出前后的尺寸一样
model.add(Conv2D(256, (1, 3), padding='same', activation="relu", name="conv1", kernel_initializer='glorot_uniform'))
model.add(Dropout(dr))
model.add(ZeroPadding2D((0, 2)))
model.add(Conv2D(80, (2, 3), padding="same", activation="relu", name="conv2", kernel_initializer='glorot_uniform'))
model.add(Dropout(dr))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_normal', name="dense1"))
model.add(Dropout(dr))
model.add(Dense( len(classes), kernel_initializer='he_normal', name="dense2" ))
model.add(Activation('softmax'))
model.add(Reshape([len(classes)]))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()


In [ ]:
nb_epoch = 100     # 使用全部数据集训练一次的次数为100
batch_size = 1024  # 每次训练一批的大小
filepath = 'convmodrecnets_CNN2_0.5.wts.h5'
history = model.fit(X_train,
    Y_train,
    batch_size=batch_size,
    epochs=nb_epoch,
    #show_accuracy=True,
    verbose=2,
    validation_data=(X_test, Y_test),
    callbacks = [
        keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto'),
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
    ]
    )
model.load_weights(filepath)

In [ ]:
score = model.evaluate(X_test, Y_test,verbose=0, batch_size=batch_size)
print (score)

In [ ]:
plt.figure()
plt.title('Training performance')
plt.plot(history.epoch, history.history['loss'], label='train loss+error')
plt.plot(history.epoch, history.history['val_loss'], label='val_error')
plt.legend()

In [ ]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues, labels=[]):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
test_Y_hat = model.predict(X_test, batch_size=batch_size)
conf = np.zeros([len(classes),len(classes)])
confnorm = np.zeros([len(classes),len(classes)])
for i in range(0,X_test.shape[0]):
    j = list(Y_test[i,:]).index(1)
    k = int(np.argmax(test_Y_hat[i,:]))
    conf[j,k] = conf[j,k] + 1
for i in range(0,len(classes)):
    confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
plot_confusion_matrix(confnorm, labels=classes)

In [ ]:
#acc = {}
#for snr in snrs:
  # test_SNRs = map(lambda x: lbl[x][1], test_idx)
  # test_X_i = X_test[np.where(np.array(test_SNRs)==snr)]
  # test_Y_i = Y_test[np.where(np.array(test_SNRs)==snr)]
  # test_X_i.shape 
  # test_Y_i.shape   
  # test_Y_i_temp = model.predict(test_X_i, batch_size=1024)
  # conf = np.zeros([len(classes),len(classes)])
  # confnorm = np.zeros([len(classes),len(classes)])
  # for i in range(0,test_X_i.shape[0]):
  #     j = list(test_Y_i[i,:]).index(1)
  #     k = int(np.argmax(test_Y_i_temp[i,:]))
  #     conf[j,k] = conf[j,k] + 1
  # for i in range(0,len(classes)):
  #     confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
  # plt.figure()
  # plot_confusion_matrix(confnorm, labels=classes, title="ConvNet Confusion Matrix (SNR=%d)"%(snr))
  # cor = np.sum(np.diag(conf))
  # ncor = np.sum(conf) - cor
  # print ("Overall Accuracy: ", cor / (cor+ncor))
  # acc[snr] = 1.0*cor/(cor+ncor)